In [1]:
import pandas as pd
import numpy as np
import re
import mysql
from mysql.connector import connect, Error


In [2]:
#define dictionary of state names to abbreviations 
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", 
               "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana",  "Maryland", "Massachusetts", "Maine", "Michigan", "Minnesota", 
               "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",  
               "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
               "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
state_names.sort()

sd = dict(zip(state_names, states))
st = dict(zip(states, state_names))

In [3]:
#set up database connection
host="162.241.224.47"
user=user
password=password
database = "easydau0_bbsiads591"

cnx = mysql.connector.connect(user=f'{user}', password=f'{password}',
                              host=f'{host}', database=f"{database}")
cursor = cnx.cursor()

In [4]:
#import user data
query  = "SELECT * FROM USERS"
users = pd.read_sql(query,cnx)

query2 = "SELECT STATE_ID, STATE_ABBR FROM US_STATES"
states_df = pd.read_sql(query2, cnx)

In [5]:
#Filter out trailing USA, remove leading cities, translate state names to abbreviations
df2=users.copy()
df2[["LOCATION"]] = df2[["LOCATION"]].replace(',\ USA$', '',regex = True)
df2[["LOCATION"]] = df2[["LOCATION"]].replace('.*,\ (?=[A-Z]{2}$)', '',regex = True)
df2[["LOCATION"]] = df2[["LOCATION"]].replace('.*\ (?=[A-Z]{2}$)', '',regex = True)
df2[["LOCATION"]] = df2[["LOCATION"]].replace('.*,\ ', '',regex = True)
df2[["LOCATION"]] = df2[["LOCATION"]].replace('.*,(?=[A-Z]{2}$)', '',regex = True)
df2[["LOCATION"]] = df2[["LOCATION"]].replace({"LOCATION": sd})

In [6]:
#filter out only US values
df3 = df2.query('LOCATION in @states')

In [7]:
df3

,AUTHOR_ID,CREATED_AT,LOCATION,DESCRIPTION,FOLLOWERS_COUNT,FOLLOWING_COUNT,LISTED_COUNT,TWEET_COUNT,VERIFIED
3,10003742,2007-11-06,NY,Anchor / Sr. Correspondent PBS @NewsHour & @Sc...,32362,1632,1002,21815,True
4,100077645,2009-12-28,CA,"All Your Favorite Entertainment News, Pop-Cult...",504344,13462,1539,62184,True
5,1000781118,2012-12-10,IN,Deathvestation is a online magazine that cover...,41,45,0,6689,False
9,100146431,2009-12-29,OK,,2623,3821,22,414807,False
12,1001796463389749248,2018-05-30,TX,,4057,699,4,137312,False
...,...,...,...,...,...,...,...,...,...
14194,996551365,2012-12-08,CA,"Writer, storyteller, latina. Periodista chilen...",878,885,28,1905,False
14195,996576014,2012-12-08,WA,amosc: wheresfreddo.,294,223,0,6783,False
14196,9967532,2007-11-05,SC,Non-native Charlestonian. @EaterCarolinas Edit...,4915,1028,132,25769,True
14204,999099590610632704,2018-05-23,PA,History History Politics,46,484,0,23055,False


In [8]:
states_dict = dict(zip(states_df['STATE_ID'], states_df['STATE_ABBR']))
user_dict = dict(zip(df3['AUTHOR_ID'], df3['LOCATION']))
final_dict = {}

for skey, sval in states_dict.items():
    for ukey, uval in user_dict.items():
        if uval == sval:
            final_dict[str(ukey)] = skey

In [9]:
final_dict

{'1102689340860387329': 1,
 '1116899440902717441': 1,
 '120596835': 1,
 '1414265682': 1,
 '14181969': 1,
 '14790432': 1,
 '155408865': 1,
 '2245857760': 1,
 '237045931': 1,
 '247822966': 1,
 '2603747575': 1,
 '261689304': 1,
 '266917386': 1,
 '27307053': 1,
 '2743144301': 1,
 '2918433237': 1,
 '307956591': 1,
 '348223061': 1,
 '377651485': 1,
 '408985764': 1,
 '426103856': 1,
 '468882192': 1,
 '482440640': 1,
 '531880032': 1,
 '548384458': 1,
 '66853888': 1,
 '721376207879725057': 1,
 '737753238892580864': 1,
 '745161748731203584': 1,
 '77391205': 1,
 '783487747780710401': 1,
 '86662923': 1,
 '873925830': 1,
 '18197096': 2,
 '185302289': 2,
 '255735722': 2,
 '40176137': 2,
 '54664525': 2,
 '946881306': 2,
 '1030714686': 3,
 '1121484839260004352': 3,
 '1175134296702517249': 3,
 '1188729405864103936': 3,
 '15237831': 3,
 '153233470': 3,
 '1549060002': 3,
 '15920054': 3,
 '167356015': 3,
 '16952753': 3,
 '1917745668': 3,
 '201134625': 3,
 '203831509': 3,
 '21579440': 3,
 '218027755': 3,
 